In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications.resnet50 import preprocess_input
import cv2
import random
import pickle
from sklearn.metrics import confusion_matrix

In [ ]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [ ]:
# Caminhos das pastas contendo as imagens de treinamento e validação
train_data_dir = "../data/subset/treino"
validation_data_dir = "../data/subset/validacao"

batch_size = 64

def preprocess_image(image):
    standardized_image = preprocess_input(image)
    return standardized_image

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    preprocessing_function=preprocess_image,
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, train_generator.num_classes), dtype=tf.float32)
    )
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    preprocessing_function=preprocess_image

)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, validation_generator.num_classes), dtype=tf.float32)
    )
)

In [ ]:
class_weights = {0: 0.5090248396342295, 1: 28.20132325141777}

In [ ]:
# Define a arquitetura do modelo
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    weights='imagenet',
    include_top=False
)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(2, activation='sigmoid')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Congela todas as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False
tf.config.run_functions_eagerly(True)  # Otherwise error

# Callbacks
cb_early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=300)

# Define o caminho para salvar os checkpoints
checkpoint_path = "../models/checkpoint"

# Callback para salvar os checkpoints
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

callbacks_list = [checkpoint_callback, cb_early_stopper]

# Define o otimizador como AdamW com os pesos de classe
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tf.keras.metrics.Recall(), 'AUC'])

# Verifica se há um checkpoint existente para carregar
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_path)
if latest_checkpoint is not None:
    print("Carregando checkpoint existente:", latest_checkpoint)
    model.load_weights(latest_checkpoint)

In [ ]:
history = model.fit(
    train_dataset,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=500,
    validation_data=validation_dataset,
    callbacks=callbacks_list,
    validation_steps=validation_generator.n // batch_size,
    class_weight = class_weights
)

In [ ]:
# Salvar o histórico de treinamento em um arquivo separado
history_path = '../models/history_mobilenetv3_weights.pickle'
with open(history_path, 'wb') as f:
    pickle.dump(history.history, f)

In [ ]:
# Plotting AUC and loss curves
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.xlabel('Época')
plt.ylabel('Erro')
plt.title('Erro de Treinamento e de Validação')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['auc'], label='Treino')
plt.plot(history.history['val_auc'], label='Validação')
plt.xlabel('Época')
plt.ylabel('AUC')
plt.title('AUC de Treinamento e de Validação')
plt.legend()
plt.show()

In [ ]:
# Evaluation and Metrics
predictions = model.predict(validation_generator)
y_true = validation_generator.classes
y_pred = predictions.argmax(axis=1)

confusion_mat = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(confusion_mat)

false_positive_rate = confusion_mat[0, 1] / (confusion_mat[0, 1] + confusion_mat[0, 0])
print("False Positive Rate:", false_positive_rate)

In [ ]:
# Obtém as classes reais e as classes previstas
y_true = validation_generator.classes
y_pred = predictions.argmax(axis=1)

# Calcula a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix with total numbers
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()

# Obtém os rótulos das classes
class_names = validation_generator.class_indices
class_names = list(class_names.keys())

tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Calculate the total numbers in each group
group_counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cm.flatten() / np.sum(cm)]

# Add the total numbers to the plot
labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_counts, group_percentages)]
labels = np.asarray(labels).reshape(cm.shape[0], cm.shape[1])

# Show the total numbers in each group
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, labels[i, j], horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de confusão')
plt.show()